In [1]:
import pandas as pd
data = pd.read_csv(r"C:\Users\Admin\Downloads\Finetune T5\C4 Dataset\C4_200M.tsv-00001-of-00010", sep='\t', header=None)


In [10]:
data.shape

(763, 2)

In [3]:
data = data[:1000]

In [5]:
#xóa các dòng bắt đầu bằng "==="
data.rename(columns={0: "error", 1: "correct"}, inplace=True)
data = data[~data['correct'].str.startswith('==')]

In [6]:
#loại các dòng trong cột error có các số lượng từ >30
data = data[data['error'].str.split().str.len() <= 30]

In [7]:
data.shape

(763, 2)

In [11]:
print(data.head())

                                               error  \
1  EverySaturday we have the King Of Club 'Drawin...   
2  Just then background of lemonade stand was ink...   
5  Using It A2 Hosting Managed WP Discount Coupon...   
6  The full “bar break” system is tensioned durin...   
7  i then completed MA in creative writing, again...   

                                             correct  
1  Every Saturday we have the King Of Clubs Drawing.  
2  Then background of the lemonade stand was inke...  
5  How to use A2 Hosting Managed WP Discount Coup...  
6  A full “bar break” system when loaded in tensi...  
7  I then completed an MA in creative writing, ag...  


In [12]:
data.rename(columns={0: "error", 1: "correct"}, inplace=True)

In [13]:
print(data.iloc[240])

error      The bell startled her from her daydream, and s...
correct    The bell startled her from her daydream, and s...
Name: 339, dtype: object


In [14]:
pd.set_option('display.max_colwidth', None)
print(data.head())
print(data.columns)

                                                                                                                                                                                   error  \
1                                                                                                                                    EverySaturday we have the King Of Club 'Drawing ...   
2        Just then background of lemonade stand was inked them Distress inks- I think they were Tumbled Glass Faded Jeans Scattered Straw Crushed Olive Mustard Seed aDd Dried Marigold.   
5                                                                                                                                  Using It A2 Hosting Managed WP Discount Coupon codes?   
6  The full “bar break” system is tensioned during load for for to-destruction, with the break outside for the influence of the connection, used together with BS 4449 graded 500 rebar.   
7                                                       i th

In [18]:
error_sentences = [str(sentence) for sentence in data["error"]]
ref_sentences = [str(sentence) for sentence in data["correct"]]

In [19]:
import re
import unicodedata
def decontracted(phrase): #hàm chuyển từ viết tắt thành từ đầy đủ
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"gon na", " going to", phrase)
    phrase = re.sub(r"wan na", " want to", phrase)
    phrase = re.sub(r"gonna", " going to", phrase)
    phrase = re.sub(r"wanna", " want to", phrase)


    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub("\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

def clean_text(t): #hàm bỏ dấu và ký tự unicode

  t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode('ascii') 
  t = decontracted(t)

  t = re.sub(r'x D', '', t)
  t = re.sub(r': D', '', t)
  t = re.sub(r': P', '', t)

  t = re.sub(r'xD', '', t)
  t = re.sub(r':D', '', t)
  t = re.sub(r':P', '', t)

  if '(' in t and ')' in t: #loại bỏ nội dung trong dấu ngoặc đơn => Hello (world) => Hello
    try:
      t = re.sub(t.split("(")[-1].split(")")[0], '', t)
    except:
      pass
    #t = re.sub("(", '', t)
    #t = re.sub(")", '', t)

  t = re.sub(r'[^A-Za-z;!?.,\-\' ]+', ' ', t) #loại bỏ các emoji, số, ký tự đặc biệt

#chuẩn hóa dấu câu, thêm khoảng trống trước dâu câu. 
  t = re.sub(r'\.+',r' .',t)
  t = re.sub(r'\;+',r' , ',t)
  t = re.sub(r'!+',r' !',t )
  t = re.sub(r'\?+',r' ?',t )
  t = re.sub(r'\-+',r' - ',t )
  t = re.sub(r'\,+',r' , ',t )
  t = re.sub(r'\'+',r" ' ",t)
  t = re.sub(' +', ' ', t)
  t = t.lower()

  return t #trả về văn bản được làm sạch

In [23]:
ref_sentences = [clean_text(sentence) for sentence in ref_sentences]
error_sentences = [clean_text(sentence) for sentence in error_sentences]

In [25]:
import transformers
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [26]:
model_path = r"C:\Users\Admin\Downloads\results\finetune_t5_latest_1_million\checkpoint-4000"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [28]:
input_text = "i want to eating banana."
inputs = tokenizer(input_text,return_tensors="pt").to(device)
outputs = model.generate(**inputs)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Câu đầu vào sai ngữ pháp:", input_text)
print("Câu sửa bởi model:", output_text)

Câu đầu vào sai ngữ pháp: i want to eating banana.
Câu sửa bởi model: i want to eat bananas .


In [29]:
outputs_sentences = []
for sentence in error_sentences[:300]:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=40)
    if len(sentence.split()) > 40:
        decoded = sentence
    else:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            preds = model.generate(**inputs, max_length=40)
        decoded = tokenizer.decode(preds[0], skip_special_tokens=True)
    outputs_sentences.append(decoded)


In [30]:
print(len(outputs_sentences))

300


In [31]:
output_sentences_formattted = []
for sentence in outputs_sentences:
    '''
    sentence = sentence.replace(","," ,")
    sentence = sentence.replace("."," .")
    sentence = sentence.replace("!"," !")
    sentence = sentence.replace("?"," ?")
    sentence = sentence.replace("  "," ")
    sentence = sentence.replace(";"," ;")
    sentence = sentence.replace(":"," :")
    '''
    sentence = sentence.replace(" -", "-")
    sentence = sentence.lower()
    output_sentences_formattted.append(sentence)

outputs_sentences = output_sentences_formattted

In [32]:
ref_sentences = [sentence.lower() for sentence in ref_sentences]
error_sentences = [sentence.lower() for sentence in error_sentences]

In [158]:
#viết câu sai vào file source.txt
with open(r"C:\Users\Admin\Downloads\errant_eva\source_c4.txt", "w", encoding="utf-8") as f:
    for sentence in error_sentences[:300]:
        f.write(sentence + "\n")
    

In [159]:
with open(r"C:\Users\Admin\Downloads\errant_eva\reference_c4.txt","w",encoding = "utf-8") as f:
    for sentence in ref_sentences[:300]:
        f.write(sentence + "\n")

with open(r"C:\Users\Admin\Downloads\errant_eva\hypothesis_c4.txt","w",encoding = "utf-8") as f:
    for sentence in outputs_sentences[:300]:
        f.write(sentence + "\n")

In [33]:
i = 39
print(f"Output sentences: {outputs_sentences[i]}")
print(f"Ref sentences: {ref_sentences[i]}")
print(f"Error sentences: {error_sentences[i]}")

Output sentences: updated postoffice and coolant handling for datron mcr posts .
Ref sentences: updated retract and coolant handling for datron mcr posts .
Error sentences: the updated postoffice and coolant handling for datron mcr posts .


In [34]:
import nltk
from nltk.util import ngrams
from collections import Counter

# Nếu chưa cài đặt bộ tokenizer:
# nltk.download('punkt')

def sentence_gleu(source: str, hypothesis: str, reference: str, max_n: int = 4) -> float:
    source_tokens = nltk.word_tokenize(source)
    hyp_tokens = nltk.word_tokenize(hypothesis)
    ref_tokens = nltk.word_tokenize(reference)
    
    overlap = 0
    total = 0

    for n in range(1, max_n + 1):
        hyp_ngrams = Counter(ngrams(hyp_tokens, n))
        ref_ngrams = Counter(ngrams(ref_tokens, n))
        src_ngrams = Counter(ngrams(source_tokens, n))

        ref_diff = ref_ngrams - src_ngrams
        hyp_diff = hyp_ngrams - src_ngrams

        match = sum((hyp_diff & ref_diff).values())
        total_hyp = sum(hyp_diff.values())

        overlap += match
        total += total_hyp

    if total == 0:
        return 1.0 if sum((ref_ngrams - src_ngrams).values()) == 0 else 0.0
    return overlap / total

def compute_corpus_gleu(sources, predictions, references, max_n=4):
    """
    Tính GLEU trung bình trên toàn bộ tập dữ liệu.
    - sources: list câu gốc (chưa sửa)
    - predictions: list câu do model sửa
    - references: list câu đúng
    """
    assert len(sources) == len(predictions) == len(references), "Danh sách không khớp độ dài"

    scores = [
        sentence_gleu(src, pred, ref, max_n=max_n)
        for src, pred, ref in zip(sources, predictions, references)
    ]
    return sum(scores) / len(scores)


In [ ]:
print(compute_corpus_gleu(error_sentences[:300], outputs_sentences[:], ref_sentences[:300]))

In [103]:
a = [1,2,3]
print(a[:3])

[1, 2, 3]


In [104]:
!pip install bert-score


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from bert_score import score
def compute_bert_score(predictions, references, lang="en"):
    """
    Tính điểm BERTScore giữa các câu dự đoán và câu tham chiếu.
    
    Args:
        predictions (list): danh sách các câu đầu ra từ mô hình.
        references (list): danh sách các câu đúng thực tế.
        lang (str): ngôn ngữ (mặc định là "en" cho tiếng Anh).
        
    Returns:
        dict: Precision, Recall, F1 trung bình và danh sách chi tiết.
    """
    P, R, F1 = score(predictions, references, lang=lang)
    
    return {
        "precision": round(P.mean().item(), 4),
        "recall": round(R.mean().item(), 4),
        "f1": round(F1.mean().item(), 4),
        "f1_list": [round(f.item(), 4) for f in F1]
    }

In [ ]:
bert_result = compute_bert_score(outputs_sentences[:300], ref_sentences[:300])
print("BERTScore F1:", bert_result["f1"])